<a href="https://colab.research.google.com/github/maisiev/Predicting-Dementia-Severity/blob/Terminal-Safe-Code/CNN_Brain_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# Run in Bash: export TF_METAL_DISABLE=1

#Force TensorFlow into single-threaded, non-XLA mode to work on MacOS
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf

tf.config.threading.set_inter_op_parallelism_threads(1)
tf.config.threading.set_intra_op_parallelism_threads(1)



import numpy as np
from tensorflow.keras import layers, Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.utils.class_weight import compute_class_weight

# =====================
# Paths and parameters
# =====================
PATH = "/content/drive/MyDrive/AI_Coursework/images/Originals"
IMG_SIZE = (224, 224)
BATCH_SIZE = 16
EPOCHS = 80
SEED = 42




In [5]:
# =====================
# Detect classes safely
# =====================
class_names = sorted([
    d for d in os.listdir(PATH)
    if os.path.isdir(os.path.join(PATH, d))
])

num_classes = len(class_names)
print("Detected classes:", class_names)
print("Number of classes:", num_classes)

assert num_classes >= 2, "You must have at least 2 classes."

Detected classes: ['MildImpairment', 'ModerateImpairment', 'NoImpairment']
Number of classes: 3


In [6]:
# =====================
# Load datasets
# =====================
train_ds = tf.keras.utils.image_dataset_from_directory(
    PATH,
    labels="inferred",
    label_mode="int",
    color_mode="grayscale",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    shuffle=True
)


val_ds = tf.keras.utils.image_dataset_from_directory(
    PATH,
    labels="inferred",
    label_mode="int",
    color_mode="grayscale",
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    shuffle=False
)

Found 5076 files belonging to 3 classes.
Using 4061 files for training.
Found 5076 files belonging to 3 classes.
Using 1015 files for validation.


In [7]:
# =====================
# Augmentation + preprocessing
# =====================
data_augmentation = Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.03),
    layers.RandomContrast(0.05),
    layers.RandomZoom(0.05)
])

def preprocess_train(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    x = data_augmentation(x)
    return x, y

def preprocess_val(x, y):
    x = tf.cast(x, tf.float32) / 255.0
    return x, y

train_ds = train_ds.map(preprocess_train)
val_ds = val_ds.map(preprocess_val)

# ⚠️ CRITICAL for macOS stability
train_ds = train_ds.prefetch(1)
val_ds = val_ds.prefetch(1)

In [8]:
# =====================
# Compute class weights
# =====================
all_labels = np.concatenate([y.numpy() for _, y in train_ds], axis=0)

classes = np.unique(all_labels)
class_weights = compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=all_labels
)

class_weight_dict = dict(zip(classes, class_weights))
print("Class weights:", class_weight_dict)

Class weights: {np.int32(0): np.float64(0.9221162579473207), np.int32(1): np.float64(2.354202898550725), np.int32(2): np.float64(0.670796167822927)}


In [10]:
# =====================
# Build CNN (dynamic output!)
# =====================
model = Sequential([
    layers.Input(shape=(224, 224, 1)),

    layers.Conv2D(32, 3, activation="relu", padding="same"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(64, 3, activation="relu", padding="same"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(128, 3, activation="relu", padding="same"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),

    layers.Conv2D(256, 3, activation="relu", padding="same"),
    layers.BatchNormalization(),
    layers.MaxPooling2D(),


    layers.GlobalAveragePooling2D(),
    layers.Dense(256, activation="relu"),
    layers.Dropout(0.3),

    layers.Dense(num_classes, activation="softmax")
])


model.compile(
    optimizer=Adam(1e-4),
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_4 (Conv2D)               │ (None, 224, 224, 32)   │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 224, 224, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 112, 112, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_5 (Conv2D)               │ (None, 112, 112, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 112, 112, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 56, 56, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 56, 56, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_6           │ (None, 56, 56, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_6 (MaxPooling2D)  │ (None, 28, 28, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 28, 28, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_7           │ (None, 28, 28, 256)    │         1,024 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_7 (MaxPooling2D)  │ (None, 14, 14, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 256)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 256)            │        65,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 3)              │           771 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 456,323 (1.74 MB)

 Trainable params: 455,363 (1.74 MB)

 Non-trainable params: 960 (3.75 KB)

In [11]:
# =====================
# Callbacks
# =====================
from tensorflow.keras.callbacks import ModelCheckpoint
early_stop = EarlyStopping(
    monitor="val_loss",
    patience=15,
    restore_best_weights=True
)

reduce_lr = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.5,
    patience=5,
    min_lr=1e-6
)
checkpoint = ModelCheckpoint(
    "best_model.h5",
    monitor="val_loss",
    save_best_only=True,
    verbose=1
)


In [ ]:


# =====================
# Train
# =====================
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=EPOCHS,
    class_weight=class_weight_dict,
    callbacks=[early_stop, reduce_lr, checkpoint]
)


# =====================
# Save model
# =====================
model.save("cnn_brain_model.h5")
print("✅ Model saved as cnn_brain_model.h5")

Epoch 1/80
  8/254 ━━━━━━━━━━━━━━━━━━━━ 12:44 3s/step - accuracy: 0.1482 - loss: 1.3624